In [1]:
def def_node_start(df,ccn='case:concept:name',cn='concept:name',ttst='time:timestamp'):
    d_node_start={}
    d_case_start={}
    for i in range (0,len(df)):
        try:
            if df[ccn][i-1]!=df[ccn][i]:
                if df[cn][i] in d_node_start.keys():
                    d_node_start[df[cn][i]]=d_node_start.get(df[cn][i])+1
                    d_case_start[df[cn][i]].append(df[ccn][i])
                else:
                    d_node_start[df[cn][i]]=1
                    d_case_start[df[cn][i]]=[df[ccn][i]]
        except:
            if df[cn][i] in d_node_start.keys():
                d_node_start[df[cn][i]]=d_node_start.get(df[cn][i])+1
                d_case_start[df[cn][i]].append(df[ccn][i])
            else:
                d_node_start[df[cn][i]]=1
                d_case_start[df[cn][i]]=[df[ccn][i]]
    return d_node_start,d_case_start

def def_edge(df,ccn='case:concept:name',cn='concept:name',ttst='time:timestamp'):
    d_edge_count = {}
    for i in range (0,len(df)):
        if not df[cn][i] in d_edge_count.keys():
            d_edge_count[df[cn][i]]={}
        try:
            if df[ccn][i]==df[ccn][i+1]:
                if df[cn][i+1] in d_edge_count[df[cn][i]].keys():
                    d_edge_count[df[cn][i]][df[cn][i+1]]=d_edge_count.get(df[cn][i]).get(df[cn][i+1])+1       
                else:
                    d_edge_count[df[cn][i]][df[cn][i+1]]=1
        except:
            pass
    return d_edge_count

def def_node_end(df,ccn='case:concept:name',cn='concept:name',ttst='time:timestamp'):
    d_node_end = {}
    d_case_end = {}
    for i in range (0,len(df)):
        try:
            if df[ccn][i]!=df[ccn][i+1]:
                if df[cn][i] in d_node_end.keys():
                    d_node_end[df[cn][i]]=d_node_end.get(df[cn][i])+1
                    d_case_end[df[cn][i]].append(df[ccn][i])
                else:
                    d_node_end[df[cn][i]]=1
                    d_case_end[df[cn][i]]=[df[ccn][i]]
        except:
            if df[cn][i] in d_node_end.keys():
                d_node_end[df[cn][i]]=d_node_end.get(df[cn][i])+1
                d_case_end[df[cn][i]].append(df[ccn][i])
            else:
                d_node_end[df[cn][i]]=1
                d_case_end[df[cn][i]]=[df[ccn][i]]
    return d_node_end,d_case_end

def def_node_count(df,ccn='case:concept:name',cn='concept:name',ttst='time:timestamp'):
    d_node_count = {}
    for i in range (0,len(df)):
        if df[cn][i] in d_node_count.keys():
            d_node_count[df[cn][i]]=d_node_count.get(df[cn][i])+1
        else:
            d_node_count[df[cn][i]]=1
    return d_node_count   
            
def def_case(df,ccn='case:concept:name',cn='concept:name',ttst='time:timestamp'):
    case = []
    for i in range (0,len(df)):
        if df[ccn][i] not in case:
            case.append(df[ccn][i])
    return case 

In [2]:
def calc(df):
    d_node_start,d_case_start=def_node_start(df)
    d_edge_count=def_edge(df)
    d_node_end,d_case_end=def_node_end(df)
    d_node_count=def_node_count(df)
    case = def_case(df)
    return (d_node_count,d_node_start,d_node_end,d_edge_count,d_case_end,d_case_start,case)

In [11]:
import os
_path = r'C:\Program Files (x86)\Graphviz2.38\bin'
os.environ["PATH"] += os.pathsep + _path
from graphviz import Digraph
import math
def draw(dict_tuple,fn = 'dot'):
    d_node_count,d_node_start,d_node_end,d_edge_count,d_case_end,d_case_start,case = dict_tuple
    f = Digraph('f',format='jpg')
    for keys in d_edge_count.keys():
        for keyst in d_edge_count.get(keys):
            if d_edge_count.get(keys).get(keyst)<=round(len(case)*0.2):
                clr='pink2'
            else:
                clr='black'
            s=str(d_edge_count.get(keys).get(keyst))
            f.edge(str(keys)+' ('+str(d_node_count.get(keys))+')', str(keyst)+' ('+str(d_node_count.get(keyst))+')', s, penwidth=str(1.04+math.log(d_edge_count.get(keys).get(keyst),10)), color=clr)
    for keys in d_node_start.keys():
        if d_node_start.get(keys)<=round(len(case)*0.2):
            clr='blue'
        else:
            clr='black'
        f.edge('start', str(keys)+' ('+str(d_node_count.get(keys))+')',str(d_node_start.get(keys)),penwidth=str(1.04+math.log(d_node_start.get(keys),10)), color=clr)
    for keys in d_node_end.keys():
        if d_node_end.get(keys)<=round(len(case)*0.2):
            clr='blue'
        else:
            clr='black'
        f.edge(str(keys)+' ('+str(d_node_count.get(keys))+')','end',str(d_node_end.get(keys)),penwidth=str(1.04+math.log(d_node_end.get(keys),10)), color=clr)
    f.node('start',shape='diamond',color='blue')
    f.node('end',shape='diamond',color='blue')
    f.save
    #print(f)
    f.render(fn,'png','t.gv')

In [12]:
import pandas as pd
import datetime as dt

In [13]:
#Считаем данные из лога, отсортируем их по дате и времени
df_full = pd.read_excel('InternationalDeclarations.xlsx')
df_full = df_full[['id-trace','concept:name','time:timestamp']]
df_full.columns = ['case:concept:name', 'concept:name', 'time:timestamp']
df_full['time:timestamp'] = df_full['time:timestamp'].apply(lambda x: pd.to_datetime(x))
df_full = df_full.sort_values(['case:concept:name','time:timestamp'], ascending=[True,True])
df_full = df_full.reset_index(drop=True)
# Расчет графа
dict_tuple_full = calc(df_full)
# Отрисовка графа
draw(dict_tuple_full,'InternationalDeclarations_full')

In [14]:
#Создадим словарь объединения событий и примапим его
_dict = {'Permit SUBMITTED by EMPLOYEE': 'Permit SUBMITTED',
 'Permit APPROVED by ADMINISTRATION': 'Permit APPROVED',
 'Permit APPROVED by BUDGET OWNER': 'Permit APPROVED',
 'Permit APPROVED by PRE_APPROVER': 'Permit APPROVED',
 'Permit APPROVED by SUPERVISOR': 'Permit APPROVED',
 'Permit FINAL_APPROVED by DIRECTOR': 'Permit FINAL_APPROVED',
 'Permit FINAL_APPROVED by SUPERVISOR': 'Permit FINAL_APPROVED',
 'Start trip': 'Start trip',
 'End trip': 'End trip',
 'Permit REJECTED by ADMINISTRATION': 'Permit REJECTED',
 'Permit REJECTED by BUDGET OWNER': 'Permit REJECTED',
 'Permit REJECTED by DIRECTOR': 'Permit REJECTED',
 'Permit REJECTED by EMPLOYEE': 'Permit REJECTED',
 'Permit REJECTED by MISSING': 'Permit REJECTED',
 'Permit REJECTED by PRE_APPROVER': 'Permit REJECTED',
 'Permit REJECTED by SUPERVISOR': 'Permit REJECTED',
 'Declaration SUBMITTED by EMPLOYEE': 'Declaration SUBMITTED',
 'Declaration SAVED by EMPLOYEE': 'Declaration SAVED',
 'Declaration APPROVED by ADMINISTRATION': 'Declaration APPROVED',
 'Declaration APPROVED by BUDGET OWNER': 'Declaration APPROVED',
 'Declaration APPROVED by PRE_APPROVER': 'Declaration APPROVED',
 'Declaration APPROVED by SUPERVISOR': 'Declaration APPROVED',
 'Declaration FINAL_APPROVED by DIRECTOR': 'Declaration FINAL_APPROVED',
 'Declaration FINAL_APPROVED by SUPERVISOR': 'Declaration FINAL_APPROVED',
 'Declaration REJECTED by ADMINISTRATION': 'Declaration REJECTED',
 'Declaration REJECTED by BUDGET OWNER': 'Declaration REJECTED',
 'Declaration REJECTED by DIRECTOR': 'Declaration REJECTED',
 'Declaration REJECTED by EMPLOYEE': 'Declaration REJECTED',
 'Declaration REJECTED by MISSING': 'Declaration REJECTED',
 'Declaration REJECTED by PRE_APPROVER': 'Declaration REJECTED',
 'Declaration REJECTED by SUPERVISOR': 'Declaration REJECTED',
 'Request Payment': 'Request Payment',
 'Payment Handled': 'Payment Handled',
 'Send Reminder': 'Send Reminder'}

df_full_gr = df_full.copy()
df_full_gr['concept:name'] = df_full_gr['concept:name'].map(_dict)
# Расчет графа
dict_tuple_full_gr = calc(df_full_gr)
# Отрисовка графа
draw(dict_tuple_full_gr,'InternationalDeclarations_full_gr')

In [90]:
# Отбираем события, которые помогут ответить на вопрос
df_full_gr_f = df_full_gr[df_full_gr['concept:name'].isin(['Permit SUBMITTED',
                                                            'Permit APPROVED',
                                                            'Permit FINAL_APPROVED',
                                                            'Declaration FINAL_APPROVED',
                                                            'Declaration APPROVED'])]
df_full_gr_f = df_full_gr_f.reset_index(drop=True)
# Расчет графа
dict_tuple_full_gr_f = calc(df_full_gr_f)
# Отрисовка графа
draw(dict_tuple_full_gr_f,'InternationalDeclarations_full_gr_isin')

In [91]:
d_node_count2,d_node_start2,d_node_end2,d_edge_count2,d_case_end2,d_case_start2,case2 = dict_tuple_full_gr_f2

In [92]:
# Проваливаемся за связь 312
df_312 = df_full_gr_f[df_full_gr_f['case:concept:name'].isin(d_case_start2['Declaration APPROVED'])]
df_312 = df_312.reset_index(drop=True)
# Выполним расчет графа
dict_tuple_312 = calc(df_312)
# Выполним отрисовку графа
draw(dict_tuple_312,'InternationalDeclarations_full_gr_isin_312')

In [94]:
# Проваливаемся за связь 116
df_116 = df_full_gr_f[df_full_gr_f['case:concept:name'].isin(d_case_start2['Declaration FINAL_APPROVED'])]
df_116 = df_116.reset_index(drop=True)
# Выполним расчет графа
dict_tuple_116 = calc(df_116)
# Выполним отрисовку графа
draw(dict_tuple_116,'InternationalDeclarations_full_gr_isin_116')
